# Competition: NBME - Score Clinical Patient Notes
## Intro
https://www.kaggle.com/c/nbme-score-clinical-patient-notes

Task: 
"[...] develop an automated method to map clinical concepts from an exam rubric (e.g., “diminished appetite”) to various ways in which these concepts are expressed in clinical patient notes written by medical students (e.g., “eating less,” “clothes fit looser”). "

If possible evaluate by a micro-averages F1-score like in the original competition

"For each instance there is a collection of ground-truth spans and a collection of predicted spans. The spans we delimit with a semicolon, like: 0 3; 5 9."

## Useful notebooks
https://www.kaggle.com/code/leemop/version-4-simply-understand-competition-itself
https://www.kaggle.com/code/drcapa/nbme-starter

relatively simple and fast:
https://www.kaggle.com/code/yasufuminakama/nbme-deberta-base-baseline-inference
https://www.kaggle.com/code/hengzheng/nbme-bert-for-beginners

transformers for the deberta one:
https://www.kaggle.com/code/yasufuminakama/nbme-pip-wheels/notebook

well commented:
https://www.kaggle.com/code/nbroad/qa-ner-hybrid-train-nbme/notebook

Roberta instead of Deberta:
https://www.kaggle.com/code/theoviel/roberta-strikes-back#Data

winner:
https://www.kaggle.com/code/currypurin/nbme-final-submit-currypurin/notebook

High scoring solutions are mostly based on the Microsoft DeBERTa model: https://github.com/microsoft/DeBERTa
An ensemble/combination of models is used to create additional pseudo-labels

Try DistilBERT (not trained from scratch, BERT is)


What is OOF?: https://stackoverflow.com/questions/52396191/what-is-oof-approach-in-machine-learning

In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [1]:
# FIXES:
# manually fixed bert.py to use collections.abc instead of sequence
from DeBERTa import deberta
import torch
class MyModel(torch.nn.Module):
  def __init__(self):
    super().__init__()
    # Your existing model code
    self.deberta = deberta.DeBERTa(pre_trained='large') # Or 'large' 'base-mnli' 'large-mnli' 'xlarge' 'xlarge-mnli' 'xlarge-v2' 'xxlarge-v2'
    # Your existing model code
    # do inilization as before
    # 
    self.deberta.apply_state() # Apply the pre-trained model of DeBERTa at the end of the constructor
    #
  def forward(self, input_ids):
    # The inputs to DeBERTa forward are
    # `input_ids`: a torch.LongTensor of shape [batch_size, sequence_length] with the word token indices in the vocabulary
    # `token_type_ids`: an optional torch.LongTensor of shape [batch_size, sequence_length] with the token types indices selected in [0, 1]. 
    #    Type 0 corresponds to a `sentence A` and type 1 corresponds to a `sentence B` token (see BERT paper for more details).
    # `attention_mask`: an optional parameter for input mask or attention mask. 
    #   - If it's an input mask, then it will be torch.LongTensor of shape [batch_size, sequence_length] with indices selected in [0, 1]. 
    #      It's a mask to be used if the input sequence length is smaller than the max input sequence length in the current batch. 
    #      It's the mask that we typically use for attention when a batch has varying length sentences.
    #   - If it's an attention mask then if will be torch.LongTensor of shape [batch_size, sequence_length, sequence_length]. 
    #      In this case, it's a mask indicate which tokens in the sequence should be attended by other tokens in the sequence. 
    # `output_all_encoded_layers`: whether to output results of all encoder layers, default, True
    encoding = deberta.bert(input_ids)[-1]

# 2. Change your tokenizer with the the tokenizer built in DeBERta
from DeBERTa import deberta
vocab_path, vocab_type = deberta.load_vocab(pretrained_id='base')
tokenizer = deberta.tokenizers[vocab_type](vocab_path)
# We apply the same schema of special tokens as BERT, e.g. [CLS], [SEP], [MASK]
max_seq_len = 512
tokens = tokenizer.tokenize('Examples input text of DeBERTa')
# Truncate long sequence
tokens = tokens[:max_seq_len -2]
# Add special tokens to the `tokens`
tokens = ['[CLS]'] + tokens + ['[SEP]']
input_ids = tokenizer.convert_tokens_to_ids(tokens)
input_mask = [1]*len(input_ids)
# padding
paddings = max_seq_len-len(input_ids)
input_ids = input_ids + [0]*paddings
input_mask = input_mask + [0]*paddings
features = {
'input_ids': torch.tensor(input_ids, dtype=torch.int),
'input_mask': torch.tensor(input_mask, dtype=torch.int)
}